In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import auc, roc_auc_score


## Download the Data

In [2]:
if not os.path.isfile("telco-churn.csv"):
    !wget -O telco-churn.csv "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv"

In [3]:
df = pd.read_csv("telco-churn.csv")
df.columns = df.columns.str.lower().str.replace(' ', '_')
categorical_columns = df.dtypes[df.dtypes == 'object'].index
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [4]:
df.totalcharges = pd.to_numeric(df['totalcharges'], errors='coerce') #change to number and set invalid to NaN
df.totalcharges = df.totalcharges.fillna(0)

In [5]:
df['churn'] = df.churn.map({'yes': 1, 'no': 0})
df['churn'].value_counts()

0    5174
1    1869
Name: churn, dtype: int64

In [6]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
len(df_train), len(df_val), len(df_test)

df_train = df_train.reset_index(drop=True)
y_train = df_train.churn
del df_train['churn']

df_val = df_val.reset_index(drop=True)
y_val = df_val.churn
del df_val['churn']

df_test = df_test.reset_index(drop=True)
y_test = df_test.churn
del df_test['churn']

In [7]:
numerical =  ['tenure', 'monthlycharges', 'totalcharges']
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
        'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [8]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [9]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:, 1]
churn_predictions = y_pred >= 0.5

In [10]:
C=1.0 
train_dicts = df_full_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts) 
y_train = df_full_train.churn
test_dicts = df_test[categorical + numerical].to_dict(orient='records')
X_test = dv.transform(test_dicts)



model = LogisticRegression(C=C)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

## Saving the model

In [11]:
import pickle

In [12]:

file_name = f"model_C={C}.bin"
file_name

'model_C=1.0.bin'

In [13]:
with open(file_name, 'wb') as f_out:
    pickle.dump((dv, model), f_out)


### Test model

In [14]:
customer = df_test.iloc[10]
customer

customerid              0111-klbqg
gender                        male
seniorcitizen                    1
partner                        yes
dependents                     yes
tenure                          32
phoneservice                   yes
multiplelines                   no
internetservice        fiber_optic
onlinesecurity                  no
onlinebackup                   yes
deviceprotection                no
techsupport                     no
streamingtv                    yes
streamingmovies                yes
contract            month-to-month
paperlessbilling               yes
paymentmethod         mailed_check
monthlycharges               93.95
totalcharges               2861.45
Name: 10, dtype: object

In [15]:
customer_dict = customer.to_dict()
customer_dict


{'customerid': '0111-klbqg',
 'gender': 'male',
 'seniorcitizen': 1,
 'partner': 'yes',
 'dependents': 'yes',
 'tenure': 32,
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'no',
 'onlinebackup': 'yes',
 'deviceprotection': 'no',
 'techsupport': 'no',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'mailed_check',
 'monthlycharges': 93.95,
 'totalcharges': 2861.45}

In [16]:
x_custom = dv.transform(customer_dict)
x_custom

array([[1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 9.39500e+01, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 3.20000e+01, 2.86145e+03]])

In [17]:
model.predict_proba(x_custom)[:, 1] >= 0.5

array([False])

In [18]:
y_test[10]

0